# DCGAN cifar10

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Convolution2D

def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128*8*8))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((128, 8, 8), input_shape=(128*8*8,)))
    model.add(UpSampling2D((2, 2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    model.add(Convolution2D(3, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    return model

Using TensorFlow backend.


In [2]:
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout

def discriminator_model():
    model = Sequential()
    model.add(Convolution2D(64, 5, 5,
                            subsample=(2, 2),
                            border_mode='same',
                            input_shape=(3, 32, 32)))
    model.add(LeakyReLU(0.2))
    model.add(Convolution2D(128, 5, 5, subsample=(2, 2)))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [3]:
# 生成画像を並べて表示するための関数
import math
import numpy as np

def combine_images(generated_images):
    total = generated_images.shape[0]
    cols = int(math.sqrt(total))
    rows = math.ceil(float(total)/cols)
    width, height = generated_images.shape[2:]
    combined_image = np.zeros((height*rows, width*cols),
                              dtype=generated_images.dtype)

    for index, image in enumerate(generated_images):
        i = int(index/cols)
        j = index % cols
        combined_image[width*i:width*(i+1), height*j:height*(j+1)] = image[0, :, :]
    return combined_image

In [ ]:
import os
from keras.datasets import cifar10
from keras.optimizers import Adam
from PIL import Image

BATCH_SIZE = 32
NUM_EPOCH = 20
GENERATED_IMAGE_PATH = 'generated_images_cifar10/' # 生成画像の保存先

def train():
    (X_train, y_train), (_, _) = cifar10.load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5

    discriminator = discriminator_model()
    d_opt = Adam(lr=1e-5, beta_1=0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)

    # generator+discriminator （discriminator部分の重みは固定）
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    g_opt = Adam(lr=2e-4, beta_1=0.5)
    dcgan.compile(loss='binary_crossentropy', optimizer=g_opt)

    num_batches = int(X_train.shape[0] / BATCH_SIZE)
    print('Number of batches:', num_batches)
    for epoch in range(NUM_EPOCH):

        for index in range(num_batches):
            noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0)

            # 生成画像を出力
            if index % 500 == 0:
                image = combine_images(generated_images)
                image = image*127.5 + 127.5
                if not os.path.exists(GENERATED_IMAGE_PATH):
                    os.mkdir(GENERATED_IMAGE_PATH)
                Image.fromarray(image.astype(np.uint8))\
                    .save(GENERATED_IMAGE_PATH+"%04d_%04d.png" % (epoch, index))

            # discriminatorを更新
            X = np.concatenate((image_batch, generated_images))
            y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)

            # generatorを更新
            noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            g_loss = dcgan.train_on_batch(noise, [1]*BATCH_SIZE)
        
        print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))
        generator.save_weights('generator_cifar10.h5')
        discriminator.save_weights('discriminator_cifar10.h5')
        
train()

Number of batches: 1562
epoch: 0, batch: 0, g_loss: 0.736224, d_loss: 0.697749
epoch: 0, batch: 1, g_loss: 0.691748, d_loss: 0.694807
epoch: 0, batch: 2, g_loss: 0.650614, d_loss: 0.695880
epoch: 0, batch: 3, g_loss: 0.582519, d_loss: 0.682803
epoch: 0, batch: 4, g_loss: 0.566267, d_loss: 0.690363
epoch: 0, batch: 5, g_loss: 0.598987, d_loss: 0.689700
epoch: 0, batch: 6, g_loss: 0.584135, d_loss: 0.695791
epoch: 0, batch: 7, g_loss: 0.605687, d_loss: 0.686038
epoch: 0, batch: 8, g_loss: 0.574314, d_loss: 0.676110
epoch: 0, batch: 9, g_loss: 0.599979, d_loss: 0.686289
epoch: 0, batch: 10, g_loss: 0.584413, d_loss: 0.679769
epoch: 0, batch: 11, g_loss: 0.530812, d_loss: 0.667412
epoch: 0, batch: 12, g_loss: 0.520698, d_loss: 0.688824
epoch: 0, batch: 13, g_loss: 0.531318, d_loss: 0.663380
epoch: 0, batch: 14, g_loss: 0.484058, d_loss: 0.663339
epoch: 0, batch: 15, g_loss: 0.501184, d_loss: 0.655606
epoch: 0, batch: 16, g_loss: 0.494332, d_loss: 0.667456
epoch: 0, batch: 17, g_loss: 0.459